In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# **SUL DATASET**

Il dataset è una versione pre-processata del dataset **OASIS Alzheimer's Detection** (un dataset di immagini MRI del cervello di 461 pazienti) utilizzato per rilevare i segni precoci dell’Alzheimer.

Poiché le immagini MRI sono 3D, sono state divise lungo l'asse z (l'asse della profondità del cervello) in 175 fette e sono state selezionate le fette che vanno dalla 55 esima alla 124 esima al fine di ridurre il numero di immagini e concentrarsi sulle sezioni cerebrali rilevanti per l'analisi della demenza; Questo approccio ha generato immagini 2D per ogni paziente.

Lo stesso approccio è stato utilizzato per le versioni delle MRI con segmentazione delle aree cerebrali offerte da OASIS effettuate tramite FSL FAST.

****
****

Le immagini sono classificate in **quattro sottoclassi**, in base alla progressione della malattia:

*   Moderate dementia (affetto da demenza)

*   Very mild dementia (affetto da demenza molto lieve)
* Mild dementia (affetto da demenza lieve)
* Non-demented (non affetto da demenza)


*La classificazione dei pazienti è stata effettuata sulla base delle valutazioni CDR (Clinical Dementia Rating), un punteggio che valuta la severità della demenza.*



In [23]:
# Import

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import zipfile
import os
import matplotlib.patches as mpatches
import numpy as np
from PIL import Image
import math
import shutil
import pandas as pd

# **OPERAZIONI SUL DATASET**

### Operazioni preliminari

In [ ]:
# Estrazione dataset

zip_path = "/content/drive/My Drive/Alzheimer-detection-project/oasis 1.zip"
extract_folder = "/content/Alzheimer-detection-project"

os.makedirs(extract_folder, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("File estratti:", os.listdir(extract_folder))


In [ ]:
# Rimozione files inutili
for dirpath, dirnames, filenames in os.walk(oasis1_path):
    for filename in filenames:
        if filename == ".DS_Store":
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
            print(f"Rimosso: {file_path}")


In [ ]:
oasis1_path = os.path.join(extract_folder, "oasis 1")
categories = ['Moderate Dementia', 'Mild Dementia', 'Very Mild Dementia', 'Non Demented']
for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_segmented_path=os.path.join(category_path, 'mri segmented')
    mri_path= os.path.join(category_path, 'mri scan')

### **VISUALIZZAZIONE GRAFICI** di distribuzione delle classi

In [ ]:
# Numero di immagini per categoria - grafico di distribuzione delle classi

somma=0

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_segmented_path=os.path.join(category_path, 'mri segmented')
    for patient in os.listdir(mri_segmented_path):
      images= os.listdir(os.path.join(mri_segmented_path, patient))
      somma=somma+len(images)
    somma=0

valori=[]

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
      images= os.listdir(os.path.join(mri_path, patient))
      somma=somma+len(images)
    print(f'numero foto {category}: {somma}')
    valori.append(somma)
    somma=0

# Visualizzazione tramite istogramma
plt.bar(categories, valori, color='skyblue')
plt.title('Distribuzione delle classi', pad=20)
plt.xlabel('Categorie')
plt.xticks(rotation=45)
plt.ylabel('Quantità foto')
plt.show()
print(f" ")

# Visualizzazione tramite grafico a torta
plt.pie(valori, labels=categories, autopct='%1.1f%%', startangle=0)
plt.title('Distribuzione delle classi',pad=20)
plt.axis('equal')
plt.show()




### **APPROCCI ALLO SBILANCIAMENTO DEL DATASET**

Il dataset è estremamente sbilanciato.

 **APPROCCI ADOTTATI PER AFFRONTARE IL PROBLEMA DELLO SBILANCIAMENTO**:

- **Unione delle due classi minoritarie** (Mild Dementia e Moderate Demenntia) dato che il focus del progetto è una early detection dell'alzheimer.
- **Downsampling parziale della classe maggioritaria** (Non Demented) per limitare il rischio di bias.
- **Bilanciamento delle classi tramiti pesi** (class weights) durante la fase di addestramento per compensare le differenze residue.

In [ ]:
# Sposto i files da Moderate dementia in Mild Dementia
mod_d_path=os.path.join(oasis1_path, "Moderate Dementia")
mild_d_path=os.path.join(oasis1_path, "Mild Dementia")
source_path=os.path.join(mod_d_path, "mri scan")
dest_path=os.path.join(mild_d_path, "mri scan")

for folder in os.listdir(source_path):
  source_file=os.path.join(source_path, folder)
  shutil.move(source_file, dest_path)

source_path=os.path.join(mod_d_path, "mri segmented")
dest_path=os.path.join(mild_d_path, "mri segmented")
for folder in os.listdir(source_path):
  source_file=os.path.join(source_path, folder)
  shutil.move(source_file, dest_path)


In [ ]:
# Rinomino la cartella
mild_to_mod_path= os.path.join(oasis1_path, "Mild to Moderate Dementia")
if os.path.exists(mild_d_path):
    os.rename(mild_d_path, mild_to_mod_path)
    print("Cartella rinominata con successo.")
else:
    print("La cartella di origine non esiste.")

categories = ['Mild to Moderate Dementia', 'Very Mild Dementia', 'Non Demented']

# Elimino la cartella vuota
cartella_da_eliminare = os.path.join(oasis1_path, "Moderate Dementia")
shutil.rmtree(cartella_da_eliminare)


In [ ]:
# Downsampling della classe maggioritaria
nd_path=os.path.join(oasis1_path, "Non Demented")
mri_path=os.path.join(nd_path, "mri scan")
mri_seg_path=os.path.join(nd_path, "mri segmented")

n=180
folders=os.listdir(mri_path)
to_delete=random.sample(folders, n)

for folder in to_delete:
  folder_seg= f"{folder.strip()} seg"
  path=os.path.join(mri_path, folder)
  path_seg=os.path.join(mri_seg_path, folder_seg)
  shutil.rmtree(path)
  shutil.rmtree(path_seg)

### **VISUALIZZAZIONE GRAFICI** di distribuzione delle classi in seguito a approcci per il bilanciamento

In [ ]:
# Numero di immagini per categoria - grafico di distribuzione delle classi
#dopo unione delle cartelle e downsampling

somma=0

valori=[]

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
      images= os.listdir(os.path.join(mri_path, patient))
      somma=somma+len(images)
    print(f'numero foto {category}: {somma}')
    valori.append(somma)
    somma=0

# Visualizzazione tramite istogramma
plt.bar(categories, valori, color='skyblue')
plt.title('Distribuzione delle classi', pad=20)
plt.xlabel('Categorie')
plt.xticks(rotation=45)
plt.ylabel('Quantità foto')
plt.show()
print(f" ")

# Visualizzazione tramite grafico a torta
plt.pie(valori, labels=categories, autopct='%1.1f%%', startangle=0)
plt.title('Distribuzione delle classi',pad=20)
plt.axis('equal')
plt.show()

### **VISUALIZZAZIONE DI IMMAGINI A CAMPIONE** dal dataset

In [ ]:
#VISUALIZZAZIONE DI IMMAGINI A CAMPIONE

# Scelta randomica della slice
#folders=os.listdir(mri_path)
#random_folder = random.choice(folders)
#patient_path= os.path.join(mri_path, random_folder)
#random_slice = random.choice(os.listdir(patient_path))

# Stampa di un immagine segmentata e un immagine T1w per categoria
for category in categories:
  category_path = os.path.join(oasis1_path, category)
  mri_segmented_path=os.path.join(category_path, 'mri segmented')
  mri_path= os.path.join(category_path, 'mri scan')
  folders=os.listdir(mri_path)
  random_folder = random.choice(folders)
  random_folder_seg= f"{random_folder.strip()} seg"

  patient_path= os.path.join(mri_path, random_folder)
  patient_path_seg= os.path.join(mri_segmented_path, random_folder_seg)

  img_path = os.path.join(patient_path, 'slice_85.png')
  img = mpimg.imread(img_path)
  img_seg_path = os.path.join(patient_path_seg, 'slice_85.png')
  img_seg = mpimg.imread(img_seg_path)

  fig, axes = plt.subplots(1, 2, figsize=(10, 5))
  axes[0].imshow(img)
  axes[0].set_title(f"Immagine T1w \n-{category}-")
  axes[0].axis('off')

  axes[1].imshow(img_seg)
  axes[1].set_title(f"Immagine segmentata \n-{category}-")
  axes[1].axis('off')

  plt.tight_layout()
  plt.show()




In [ ]:
# Visualizzazione con etichette delle immagini segmentate

# Scelta di un immagine segmentata (parte commentata per randomizzazione)

#category='Very Mild Dementia'
#category_path = os.path.join(oasis1_path, category)
#mri_segmented_path = os.path.join(category_path, 'mri segmented')
#folders = os.listdir(mri_segmented_path)
#random_folder = random.choice(folders)
#patient_path = os.path.join(mri_segmented_path, random_folder)
#random_slice = random.choice(os.listdir(patient_path))
#img_seg_path = os.path.join(patient_path, random_slice)

img_seg = mpimg.imread(img_seg_path)
img_seg = img_seg[:, :, 0]

labels=['Background', 'CSF', 'Materia grigia', 'Materia bianca']
valori_unici = np.unique(img_seg)
print(f"{len(valori_unici)} valori differenti: {valori_unici}\n\n")

# creazione della leggenda e visualizzazione
cmap = plt.cm.Set1_r
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(img_seg, cmap=cmap)

patches = []
for i, val in enumerate(valori_unici):
  patch = mpatches.Patch(color=cmap(val), label=labels[i])
  patches.append(patch)
ax.legend(handles=patches, loc='upper right')

ax.axis('off')
plt.tight_layout()
plt.show()

I valori dell'output segmentato sono normalizzati e lo standard è:

| Valore         | Etichetta         |  
|----------------|-------------------|
| `0.0`          | **Background**     |
| `0.33333334`   | **CSF**            |
| `0.6666667`    | **Materia Grigia** |
| `1.0`          | **Materia Bianca** |

### **OPERAZIONI SU DIMENSIONI E CANALI** delle immagini

In [ ]:
# Check dimensioni e canali

images_dimentions=set()
images_channels=set()

# Per ognni immagine in ogni categoria salva dimensioni e numero di canale negli insiemi
for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
      patient_path=os.path.join(mri_path, patient)
      file_path=os.path.join(patient_path, '.DS_Store')
      if os.path.isfile(file_path):
        os.remove(file_path)
      images= os.listdir(os.path.join(mri_path, patient))

      for image in images:
        image_path=os.path.join(patient_path, image)
        i = mpimg.imread(image_path)
        w, h = i.shape[:2]
        images_dimentions.add((w, h))

        if len(i.shape) == 3:
          channels = i.shape[2]
        else:
          channels = 1
        images_channels.add(channels)

# Controllo del numero di elementi negli insiemi
if len(images_dimentions) == 1:
  print("tutte le immagini hanno le stesse dimensioni", images_dimentions.pop())
else:
  print("NON tutte le immagini hanno le stesse dimensioni:")
  for dim in images_dimentions:
        print(" ", dim)

if len(images_channels) == 1:
    print("tutte le immagini hanno lo stesso numero di canali:", images_channels.pop())
else:
    print("NON tutte le immagini hanno lo stesso numero di canali:")
    for ch in images_channels:
        print(" ", ch)



In [ ]:
# Conversione delle immagini MRI in scale di grigi
# Utilizzo Grayscale = 0.299 * R + 0.587 * G + 0.114 * B

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
        patient_path = os.path.join(mri_path, patient)
        file_path = os.path.join(patient_path, '.DS_Store')
        if os.path.isfile(file_path):
            os.remove(file_path)
        images = os.listdir(os.path.join(mri_path, patient))

        for image in images:
            image_path = os.path.join(patient_path, image)
            i = mpimg.imread(image_path)

            if len(i.shape) == 3 and i.shape[2] >= 3:
                i_rgb = i[..., :3]

                i_gray = 0.299 * i_rgb[..., 0] + 0.587 * i_rgb[..., 1] + 0.114 * i_rgb[..., 2]
                i_gray = (i_gray * 255).astype(np.uint8)
                gray_img = Image.fromarray(i_gray)
                gray_img.save(image_path)

# Check del nuovo numero di canali
images_channels=set()
for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
      patient_path=os.path.join(mri_path, patient)
      images= os.listdir(os.path.join(mri_path, patient))

      for image in images:
        image_path=os.path.join(patient_path, image)
        i = mpimg.imread(image_path)

        if len(i.shape) == 3:
          channels = i.shape[2]
        else:
          channels = 1
        images_channels.add(channels)

if len(images_channels) == 1:
    print("tutte le immagini hanno lo stesso numero di canali:", images_channels.pop())
else:
    print("NON tutte le immagini hanno lo stesso numero di canali:")
    for ch in images_channels:
        print(" ", ch)

### **DEVIAZIONE STANDARD E MSE**

 **DEVIAZIONE STANDARD --> misura dove variano di più i pixel**

- STD ALTA: molta variabilità nei valori dei pixel

- STD BASSA: poca variabilità nei valori dei pixel

**MEAN SQUARE ERROR --> misura quanto ogni pixel si allontana dalla media**

- MSE ALTO: alta variabilità intra-classe

- MSE BASSO: bassa variabilità intra-classe

In [ ]:
# Calcolo della deviazione standard e mse

std_interclasse= {}
mse_interclasse={}
std_intraclasse={}
mse_intraclasse={}


selected_slices = ['slice_60.png', 'slice_85.png', 'slice_103.png', 'slice_120.png']  # Esempio: nomi delle slice

for name in selected_slices:  #for name in images per prendere tutte le immagini
  slices_all_categories = []
  for category in categories:
    slices = []
    category_path = os.path.join(oasis1_path, category)
    mri_path= os.path.join(category_path, 'mri scan')
    for patient in os.listdir(mri_path):
      patient_path=os.path.join(mri_path, patient)
      slice_of_interest= os.path.join(patient_path, name)
      i = Image.open(slice_of_interest)
      i_array = np.array(i)
      slices.append(i_array)

    if slices:
        slices_stack_interclasse = np.stack(slices)
        std_map_interclasse = np.std(slices_stack_interclasse, axis=0)
        mean_slice_interclasse = np.mean(slices_stack_interclasse, axis=0)
        mse_map_interclasse = np.mean((slices_stack_interclasse - mean_slice_interclasse) ** 2, axis=0)
        mse_interclasse[(name,category)] = mse_map_interclasse
        std_interclasse[(name, category)] = std_map_interclasse

        slices_all_categories.extend(slices)

  if slices_all_categories:
        slices_stack_intraclasse = np.stack(slices_all_categories)
        std_map_intraclasse = np.std(slices_stack_intraclasse, axis=0)
        mean_slice_intraclasse = np.mean(slices_stack_intraclasse, axis=0)
        mse_map_intraclasse = np.mean((slices_stack_intraclasse - mean_slice_intraclasse) ** 2, axis=0)
        std_intraclasse[name] = std_map_intraclasse
        mse_intraclasse[name] = mse_map_intraclasse





In [ ]:
#  Visualizzazione della deviazione standard (sulla stessa slice) intra classe

columns = 3
imgs = len(std_interclasse)
rows = math.ceil(imgs / columns)
plt.figure(figsize=(columns * 3, rows * 3))

for idx, ((name, category), std_map_interclasse) in enumerate((std_interclasse.items())):
    plt.subplot(rows, columns, idx + 1)
    im = plt.imshow(std_map_interclasse, cmap='hot')
    plt.title(f"STD - {category}\n{name}")
    plt.axis('off')

plt.tight_layout()
plt.colorbar(im, ax=plt.gcf().get_axes(), fraction=0.02, pad=0.04)
plt.show()

In [ ]:
# Visualizzazione della deviazione standard (sulla stessa slice) inter classe

columns = 4
imgs = len(std_intraclasse)
rows = math.ceil(imgs / columns)
plt.figure(figsize=(columns * 3, rows * 3))

for idx, (name, std_map_intraclasse) in enumerate(std_intraclasse.items()):
    plt.subplot(rows, columns, idx + 1)
    im = plt.imshow(std_map_intraclasse, cmap='hot')
    plt.title(f"STD - {name}")
    plt.axis('off')

plt.tight_layout()
plt.colorbar(im, ax=plt.gcf().get_axes(), fraction=0.02, pad=0.04)
plt.show()

In [ ]:
# Visualizzazione dell'MSE (sulla stessa slice) intra classe

columns = 3
imgs = len(mse_interclasse)
rows = math.ceil(imgs / columns)
plt.figure(figsize=(columns * 3, rows * 3))

for idx, ((name, category), mse_map_interclasse) in enumerate(mse_interclasse.items()):
    plt.subplot(rows, columns, idx + 1)
    im = plt.imshow(mse_map_interclasse, cmap='hot')
    plt.title(f"MSE - {category}\n{name}")
    plt.axis('off')

plt.tight_layout()
plt.colorbar(im, ax=plt.gcf().get_axes(), fraction=0.02, pad=0.04)
plt.show()

In [ ]:
# Visualizzazione dell'MSE (sulla stessa slice) inter classe

columns = 4
imgs = len(mse_intraclasse)
rows = math.ceil(imgs / columns)
plt.figure(figsize=(columns * 3, rows * 3))

for idx, (name, mse_map_intraclasse) in enumerate(mse_intraclasse.items()):
    plt.subplot(rows, columns, idx + 1)
    im = plt.imshow(mse_map_intraclasse, cmap='hot')
    plt.title(f"MSE - {name}")
    plt.axis('off')

plt.tight_layout()
plt.colorbar(im, ax=plt.gcf().get_axes(), fraction=0.02, pad=0.04)
plt.show()

### **DIVISIONE IN TRAINING SET E TEST SET**

In [ ]:
# Divisione in training e test set

labeled_patients=[]
segmented_patients=[]

for cat in categories:
  category_path = os.path.join(oasis1_path, cat)
  mri_path= os.path.join(category_path, 'mri scan')
  mri_segmented_path=os.path.join(category_path, 'mri segmented')
  for patient in os.listdir(mri_path):
    patient_seg= f"{patient.strip()} seg"
    patient_path=os.path.join(mri_path, patient)
    patient_seg_path=os.path.join(mri_segmented_path, patient_seg)
    labeled_patients.append((cat, patient))
    segmented_patients.append((cat, patient_seg))

df=pd.DataFrame(labeled_patients, columns=["label", "patient_id"])
df_seg=pd.DataFrame(segmented_patients, columns=["label", "patient_id"])


In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

train_ids = train_x['patient_id'].tolist()
test_ids = test_x['patient_id'].tolist()

train_y = df_seg[df_seg['patient_id'].isin([f"{pid} seg" for pid in train_ids])]
test_y = df_seg[df_seg['patient_id'].isin([f"{pid} seg" for pid in test_ids])]

print("Numero di pazienti segmentati in train_y:", len(train_y))
print("Numero di pazienti in train_x:", len(train_x))
print("Numero di pazienti segmentati in test_y:", len(test_y))
print("Numero di pazienti in test_x:", len(test_x))


### **CONVERSIONE IMMAGINI SEGMENTATE IN MAPPE DI CLASSI**

In [ ]:
# Funzione per la conversione di immagine segmentata in una mappa di classi intere
value_to_class = {
    0.0: 0,
    1/3: 1,
    2/3: 2,
    1.0: 3
}

def class_map(img, d):
  class_map = np.zeros_like(img, dtype=np.uint8)
  for val, class_id in d.items():
    mask = np.isclose(img, val)
    class_map[mask] = class_id
  return class_map

In [ ]:
# Conversione delle immagini segmentate in mappe

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_segmented_path=os.path.join(category_path, 'mri segmented')
    for patient in os.listdir(mri_segmented_path):
      patient_path= os.path.join(mri_segmented_path, patient)
      for filename in os.listdir(patient_path):
        img_path = os.path.join(patient_path, filename)
        img_seg = mpimg.imread(img_seg_path)
        img_seg = img_seg[:, :, 0]
        classmap = class_map(img_seg, value_to_class)
        save_path = os.path.join(patient_path, filename.replace(".png", ".npy"))
        np.save(save_path, classmap)

### **SBILANCIAMENTO TRA CLASSI DELLA SEGMENTAZIONE**

In [ ]:
# Calcolo del numero di pixel per ogni classe per vedere se sono presenti sbilanciamenti

labels = ['Background', 'CSF', 'Materia grigia', 'Materia bianca']
total_counts = np.zeros(4, dtype=np.int64)
image_count = 0

for category in categories:
    category_path = os.path.join(oasis1_path, category)
    mri_segmented_path=os.path.join(category_path, 'mri segmented')
    for patient in os.listdir(mri_segmented_path):
      patient_path= os.path.join(mri_segmented_path, patient)
      for filename in os.listdir(patient_path):
        if not filename.endswith('.npy'):
          continue
        file_path = os.path.join(patient_path, filename)
        class_map = np.load(file_path)
        pixel_counts = np.bincount(class_map.flatten(), minlength=4)
        total_counts += pixel_counts
        image_count += 1
avg_counts = total_counts / image_count


# Visualizzazione tramite istogramma
plt.bar(labels, total_counts)
plt.xlabel('Classi')
plt.ylabel('Numero di Pixel')
plt.title('Distribuzione dei Pixel per Classe')
plt.show()

# Visualizzazione tramite grafico a torta
plt.pie(total_counts, labels=labels, autopct='%1.1f%%', startangle=0)
plt.title('Distribuzione dei pixel per classe',pad=20)
plt.axis('equal')
plt.show()

In [ ]:
# Calcolo dei pesi per la loss function

pixels = avg_counts.sum()
class_weights = pixels / (4 * avg_counts)

for class_id, weight in enumerate(class_weights):
    print(f"Classe {labels[class_id]}: Peso = {weight:.4f}")

# **DEFINIZIONE DELLA U NET** per la segmentazione

## *Funzioni ausiliarie al forward pass*

### **Funzioni di pooling e upsampling**

In [122]:
# Max pooling

def max_pooling(img, pool_size=2, stride=2):
  if img.ndim == 3:
    img = img[:, :, :, np.newaxis]
  batch_size, img_height, img_width, img_channels = img.shape
  pool_height, pool_width = (pool_size, pool_size)
  output_height = (img_height - pool_height) // stride + 1
  output_width = (img_width - pool_width) // stride + 1
  output = np.zeros((batch_size, output_height, output_width, img_channels))
  for b in range(batch_size):
    for i in range(output_height):
      for j in range(output_width):
        row=i*stride
        column=j*stride
        for c in range(img_channels):
          window = img[b, row:row + pool_height, column:column + pool_width, c]
          output[b,i, j, c] = np.max(window)
  return output

In [123]:
# Upsampling

def upsample(img, scale=2):
  if img.ndim == 3:  # (B, H, W)
    img = img[:, :, :, np.newaxis]
  batch_size, img_height, img_width, img_channels = img.shape
  output_height=img_height*scale
  output_width=img_width*scale
  output = np.zeros((batch_size, output_height, output_width, img_channels))
  for b in range(batch_size):
    for row in range(img_height):
      for column in range(img_width):
        for i in range(scale):
          for j in range(scale):
            for c in range(img_channels):
              output[b, row * scale + i, column * scale + j, c] = img[b,row, column,c]
  return output

### **Funzione di perdita** (loss function)

In [124]:
# Loss function e funzioni ausiliarie

def onehot_encoding(targets, n_classes):
  identity = np.eye(n_classes)
  output = identity[targets]


  return output



def weighted_cross_entropy_loss(targets, predictions, class_weights):
  epsilon = 1e-10
  log_predictions = np.log(predictions + epsilon)

  one_hot_targets = onehot_encoding(targets, n_classes)
  cross_entropy = -np.sum(one_hot_targets * log_predictions, axis=-1)
  weighted_cross_entropy = cross_entropy * class_weights[targets]
  loss_per_batch = np.sum(weighted_cross_entropy, axis=(1, 2))
  loss = np.mean(loss_per_batch)

  return loss



### **Funzioni di attivazione**

In [125]:
# ReLU

def ReLU(img):
  return np.maximum(0, img)

In [126]:
# Softmax - per rendere l'output in probabilità per classe

def softmax(predictions):
  if predictions.ndim != 4:
    raise ValueError(f"Expected predictions of shape (B, H, W, C), got {predictions.shape}")
  exp_preds = np.exp(predictions - np.max(predictions, axis=1, keepdims=True))
  sum_exp_preds = np.sum(exp_preds, axis=1, keepdims=True)
  softmax_output = exp_preds / sum_exp_preds
  return softmax_output

### **Funzioni di ottimizzazione**

## *Funzioni per il forward pass*

In [127]:
# Convoluzione

def convolve(imgs, kernel):
  if imgs.ndim == 3:
    imgs = imgs[:, :, :, np.newaxis]

  batch_size, img_height, img_width, imgs_channels = imgs.shape
  ker_height, ker_width = kernel.shape

  pad_h = ker_height // 2
  pad_w = ker_width // 2
  padded_imgs = np.pad(imgs, pad_width=((0, 0), (pad_h, pad_h), (pad_w, pad_w), (0, 0)),
        mode='constant',constant_values=0)

  output = np.zeros((batch_size, img_height, img_width, imgs_channels))
  for i in range(batch_size):
    img = padded_imgs[i]
    for row in range(img_height):
      for column in range(img_width):
        for c in range(imgs_channels):
          window = img[row:row + ker_height, column:column + ker_width,c]
          output[i, row, column, c]=np.sum(window * kernel)
  return output

In [128]:
# Convoluzione finale

def final_convolve_1x1(imgs, filters):
  img_batch, img_height, img_width, img_channels = imgs.shape
  _, _, _, filters_channels = filters.shape

  output = np.zeros((img_batch, img_height, img_width, filters_channels))
  for b in range(img_batch):
    for h in range(img_height):
      for w in range(img_width):
        for oc in range(filters_channels):
          for ic in range(img_channels):
            output[b, h, w, oc] += imgs[b, h, w, ic] * filters[0, 0, ic, oc]
  return output

In [129]:
# Funzione per la concatenazione

def concatenate(img_contr, img_exp):
  if img_contr.ndim == 3:
    img_contr = img_contr[:, :, :, np.newaxis]
  if img_exp.ndim == 3:
        img_exp = img_exp[:, :, :, np.newaxis]
  batch_size, contr_height, contr_width , _= img_contr.shape
  _, exp_height, exp_width, _ = img_exp.shape
  if contr_height == exp_height and contr_width == exp_width:
    concatenated = np.concatenate((img_contr, img_exp), axis=3)
    return concatenated

In [130]:
# Encoder

def encoder(imgs, filters):
  blocks=[]
  saved_enc = {}
  saved_enc["input_convolve"] = imgs

  c=convolve(imgs, filters[0])
  saved_enc["conv0"] = c
  c=ReLU(c)
  blocks.append(c)

  c=convolve(c, filters[1])
  saved_enc["conv1"] = c
  c=ReLU(c)
  blocks.append(c)

  c=max_pooling(c)

  return c, blocks, saved_enc

In [131]:
# Decoder

def decoder(imgs, blocks, filters):
  saved = {}

  saved["input_upsample"] = imgs
  c = upsample(imgs)
  saved["upsampled"] = c

  skip = blocks.pop()
  saved["skip"] = skip

  c = concatenate(c, skip)
  saved["concatenated"] = c

  c = convolve(c, filters[0])
  saved["conv0"] = c
  c = ReLU(c)

  saved["input_conv1"] = c
  c = convolve(c, filters[1])
  saved["conv1"] = c
  c = ReLU(c)

  return c, saved


In [146]:
# Bottleneck

def bottleneck(imgs, filters):
  saved_b = {}

  saved_b["input_convolve0"] = imgs
  c=convolve(imgs, filters[0])
  saved_b["convolved0"] = c
  c=ReLU(c)

  saved_b["input_convolve1"] = c
  c=convolve(c, filters[1])
  saved_b["convolved1"] = c
  c=ReLU(c)

  return c , saved_b

## *Funzioni ausiliarie al backward pass*

### **Funzioni di pooling e upsampling**

In [133]:
# Derivata del max pooling

def max_pooling_backward(dout, input, pool_size, stride):

  batch_size, height, width, channels = input.shape
  out_height, out_width = dout.shape[1], dout.shape[2]

  output = np.zeros_like(input)

  for i in range(batch_size):
    for h in range(out_height):
      for w in range(out_width):
        for c in range(channels):
          h_start = h * stride
          h_end = h_start + pool_size
          w_start = w * stride
          w_end = w_start + pool_size

          window = input[i, h_start:h_end, w_start:w_end, c]
          max_pos = np.unravel_index(np.argmax(window), window.shape)

          output[i, h_start + max_pos[0], w_start + max_pos[1], c] += dout[i, h, w, c]

  return output


In [134]:
# Derivata dell'upsampling

def upsample_backward(dout, img, scale=2):
  if img.ndim == 3:
      img = img[:, :, :, np.newaxis]
  batch_size, output_height, output_width, img_channels = dout.shape

  img_height = output_height // scale
  img_width = output_width // scale

  # Creiamo una variabile di output per accumulare i gradienti
  dinput = np.zeros_like(img)

  for b in range(batch_size):
    for row in range(img_height):
      for column in range(img_width):
        for i in range(scale):
          for j in range(scale):
            for c in range(img_channels):
              dinput[b, row, column, c] += dout[b, row * scale + i, column * scale + j, c]

  return dinput


### **Funzioni di attivazione**

In [135]:
# Derivata della ReLU

def ReLU_backward(grad, input):
  return grad * (input > 0)

### **Funzioni di perdita** (loss function)

In [136]:
# Gradiente della funzione di loss
def grad(targets, predictions, class_weights):
  one_hot_targets = onehot_encoding(targets, predictions.shape[-1])
  weights = class_weights[targets]
  weights = weights[..., np.newaxis]
  grad = - (one_hot_targets * weights) / (predictions + 1e-8)
  grad /= np.prod(targets.shape)
  return grad

## *Funzioni per il backward pass*

In [137]:
# Derivata della convoluzione

def convolve_backward(dout, input, kernel):
  if input.ndim == 3:
    input = input[:, :, :, np.newaxis]
  if dout.ndim == 3:
    dout = dout[:, :, :, np.newaxis]

  batch_size, img_height, img_width, channels = input.shape
  ker_heigt, ker_width = kernel.shape

  pad_h = ker_heigt // 2
  pad_w = ker_width // 2

  padded_input = np.pad(input, ((0, 0), (pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant')
  dinput = np.zeros_like(padded_input, dtype=np.float64)
  dkernel = np.zeros_like(kernel, dtype=np.float64)

  for b in range(batch_size):
    for h in range(img_height):
      for w in range(img_width):
        for c in range(channels):
          window = padded_input[b, h:h + ker_heigt, w:w + ker_width, c]
          dkernel += window * dout[b, h, w, c]
          dinput[b, h:h + ker_heigt, w:w + ker_width, c] += kernel * dout[b, h, w, c]

  dinput = dinput[:, pad_h:pad_h + img_height, pad_w:pad_w + img_width, :]
  return dinput, dkernel



In [138]:
# Derivata della convoluzione finale

def final_convolve_1x1_backward(dout, imgs, filters):
  img_batch, img_height, img_width, img_channels = imgs.shape
  _, _, _, filters_channels = filters.shape

  dimgs = np.zeros_like(imgs)
  dfilters = np.zeros_like(filters)

  for b in range(img_batch):
    for h in range(img_height):
      for w in range(img_width):
        for oc in range(filters_channels):
          for ic in range(img_channels):
            dimgs[b, h, w, ic] += dout[b, h, w, oc] * filters[0, 0, ic, oc]
            dfilters[0, 0, ic, oc] += imgs[b, h, w, ic] * dout[b, h, w, oc]

    return dimgs, dfilters



In [139]:
# Derivata della concatenazione

def concatenate_backward(dout, img_contr, img_exp):
  c1 = img_contr.shape[-1]
  c2 = img_exp.shape[-1]
  dimg_contr = dout[..., :c1]
  dimg_exp = dout[..., c1:c1 + c2]
  return dimg_contr, dimg_exp


In [150]:
# Encoder backward

def encoder_backward(dout, blocks, filters, saved_enc):
  grads = [None, None]

  dmaxpool = max_pooling_backward(dout, blocks.pop(), pool_size=2, stride=2)

  dout = ReLU_backward(dmaxpool, saved_enc["conv1"])
  dconv1, grads[1] = convolve_backward(dout, blocks.pop(), filters[1])

  dout = ReLU_backward(dconv1, saved_enc["conv0"])
  dconv2, grads[0] = convolve_backward(dout, saved_enc["input_convolve"], filters[0])

  return dconv2, grads


In [148]:
# Decoder backwards

def decoder_backward(dout, saved, filters):
  grads = [None, None]

  drelu2 = ReLU_backward(dout, saved["conv1"])

  dconv2, grads[1] = convolve_backward(drelu2, saved["input_conv1"], filters[1])

  drelu1 = ReLU_backward(dconv2, saved["conv0"])

  dconv1, grads[0] = convolve_backward(drelu1, saved["concatenated"], filters[0])

  dupsampled, dskip = concatenate_backward(dconv1, [saved["upsampled"], saved["skip"]])

  dinput = upsample_backward(dupsampled, saved["input_upsample"])

  return dinput, dskip, grads


In [147]:
# Bottleneck backwards

def decoder_backward(dout, saved, filters):
  grads = [None, None]

  drelu2 = ReLU_backward(dout, saved["convolved1"])
  dconv2, grads[1] = convolve_backward(drelu2, saved["input_convolve1"], filters[1])

  drelu1 = ReLU_backward(dout, saved["convolved0"])
  dconv1, grads[0] = convolve_backward(drelu2, saved["input_convolve0"], filters[1])

  return dconv1, grads

## FUNZIONI DI TEST

In [142]:
# FORWARD
def forward_pass(input_img, target_img, filters, class_weights, n_classes):
  print("Input immagine:", input_img.shape)

  encoded_img, skip_connections, saved_encoder = encoder(input_img, filters)
  print("Output dell'encoder:", encoded_img.shape)

  bottleneck_img = bottleneck(encoded_img, filters)
  print("Output del bottleneck:", bottleneck_img.shape)

  decoded_img, saved_decoder = decoder(bottleneck_img, skip_connections, filters)
  print("Output del decoder:", decoded_img.shape)

  in_channels = decoded_img.shape[-1]
  final_filters = np.random.rand(1, 1, in_channels, n_classes)
  final_output = final_convolve_1x1(decoded_img, final_filters)
  print("Output finale dopo convoluzione 1x1:", final_output.shape)

  loss = weighted_cross_entropy_loss(target_img, final_output, class_weights)
  print("Loss:", loss)

  return final_output, loss

In [143]:
# BACKWARD



In [145]:
# test con immagini e filtri random

batch_size = 10
img_height, img_width = 176, 208
n_classes = 4
class_weights = np.random.rand(n_classes)


imgs = np.random.rand(batch_size, img_height, img_width, 1)
targets = np.random.randint(0, n_classes, size=(batch_size, img_height, img_width))

filters = np.random.rand(batch_size,3, 3)
filters = np.array(filters)

output, loss = forward_pass(imgs, targets, filters, class_weights, n_classes)

Input immagine: (10, 176, 208, 1)
Output dell'encoder: (10, 88, 104, 1)
Output del bottleneck: (10, 88, 104, 1)
Output del decoder: (10, 176, 208, 2)
Output finale dopo convoluzione 1x1: (10, 176, 208, 4)
Loss: -135238.9005660288
